<a href="https://colab.research.google.com/github/ggwnsghgg/book_project/blob/main/%EC%B1%85_%EC%B6%94%EC%B2%9C%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install pymysql
!pip install konlpy

In [50]:
import pymysql
import pandas as pd
import numpy as np
import konlpy

#데이터 가져오기
db = pymysql.connect(host='220.86.191.27', port=3306, user='playdata', password='playdata123!', db='bookdata', charset='utf8')

curs = db.cursor(pymysql.cursors.DictCursor)

query = '''
select *
    from books_data'''
df_books_data = pd.read_sql(query, db)

In [51]:
#스토리의 키워드를 뽑기위해 명사와 형용사만 가져오자
story_words = df_books_data['story'].to_numpy()
for i, document in enumerate(story_words):
    okt = konlpy.tag.Okt()
    clean_words = []
    for word in okt.pos(document, stem = True): #어간 뽑자
        if word[1] in ['Noun', 'Adjective']: #명사, 형용사만 가져와
            clean_words.append(word[0])
    #print(clean_words)
    document = ' '.join(clean_words)
    #print(document)
    story_words[i] = document

In [52]:
#넘파이 배열 데이터프레임 화
df_story_words=pd.DataFrame(story_words,
            columns = ['story_words'])

In [53]:
df_books = pd.concat([df_books_data, df_story_words], axis = 1)

In [54]:
df_books.columns

Index(['id', 'title', 'author', 'pub', 'date', 'price', 'review', 'rating',
       'story', 'img', 'genre', 'story_words'],
      dtype='object')

In [55]:
#날짜, 아이디 날리고
df_books = df_books.drop(['id','date'],axis = 1)

In [56]:
#혹시모를 중복 제거하자
df_books = df_books.drop_duplicates(['title'])

In [57]:
df_books.tail()

,title,author,pub,price,review,rating,story,img,genre,story_words
79768,그녀를 위해,,신영미디어,2880,0,0.0,마르코 고아 조카 안젤리나 사람 필요하다 양육 위 편의 상의 아내 성실하다 착하다 ...,http://image.yes24.com/goods/373089/L,로맨스,마르코 고아 조카 안젤리나 사람 필요하다 양육 위 편의 상의 아내 성실하다 착하다 ...
79769,하루만 사랑해 1,,파피루스(디앤씨미디어),8100,1,7.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,http://image.yes24.com/goods/375482/L,로맨스,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...
79770,하루만 사랑해 2,,파피루스(디앤씨미디어),8100,0,0.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,http://image.yes24.com/goods/375484/L,로맨스,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...
79771,서울 여자와 대구 남자가 만날 때,정민,영상노트,8100,0,0.0,,http://image.yes24.com/goods/376458/L,로맨스,
79772,kiss me...,,현무,7200,2,5.0,,http://image.yes24.com/goods/377351/L,로맨스,


In [58]:
df_books = df_books.dropna(axis=0)

In [59]:
df_books

,title,author,pub,price,review,rating,story,img,genre,story_words
0,요리코를 위해,노리즈키 린타로,모모,13500,43,9.2,스스로 악마 이자 신 딸 살해 남자 심판 아버지 비극 출발 이자 마지막 끝내 실체 ...,http://image.yes24.com/goods/89392792/L,추리/미스터리,스스로 악마 이자 신 딸 살해 남자 심판 아버지 비극 출발 이자 마지막 끝내 실체 ...
1,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,14220,13,9.4,하나 보석 점 살인 사건 과거 미래 사건 경쾌하다 리듬 감 있다 터치 게이고 복고 ...,http://image.yes24.com/goods/97561319/L,추리/미스터리,하나 보석 점 살인 사건 과거 미래 사건 경쾌하다 리듬 감 있다 터치 게이고 복고 ...
2,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),16200,54,9.1,아버지 살해 당하다 그것 결혼 소식 직후 행복하다 순간 비극 시작 대기업 취직 후 ...,http://image.yes24.com/goods/95712763/L,추리/미스터리,아버지 살해 당하다 그것 결혼 소식 직후 행복하다 순간 비극 시작 대기업 취직 후 ...
3,돌이킬 수 없는 약속,야쿠마루 가쿠,북플라자,13500,217,8.2,제 에드가 란 포상 수상 작가 야쿠 마루 가쿠 미스터리 추리 소설 내 딸 살해 놈 ...,http://image.yes24.com/goods/35552641/L,추리/미스터리,제 에드가 란 포상 수상 작가 야쿠 마루 가쿠 미스터리 추리 소설 내 딸 살해 놈 ...
4,루팡의 딸,요코제키 다이,북플라자,13500,31,8.5,도둑 집안 대부 변 사체 발견 도둑 집안 대부 이 오 하나 코 손녀 딸 있다 하나 ...,http://image.yes24.com/goods/78860364/L,추리/미스터리,도둑 집안 대부 변 사체 발견 도둑 집안 대부 이 오 하나 코 손녀 딸 있다 하나 ...
...,...,...,...,...,...,...,...,...,...,...
79768,그녀를 위해,,신영미디어,2880,0,0.0,마르코 고아 조카 안젤리나 사람 필요하다 양육 위 편의 상의 아내 성실하다 착하다 ...,http://image.yes24.com/goods/373089/L,로맨스,마르코 고아 조카 안젤리나 사람 필요하다 양육 위 편의 상의 아내 성실하다 착하다 ...
79769,하루만 사랑해 1,,파피루스(디앤씨미디어),8100,1,7.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,http://image.yes24.com/goods/375482/L,로맨스,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...
79770,하루만 사랑해 2,,파피루스(디앤씨미디어),8100,0,0.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,http://image.yes24.com/goods/375484/L,로맨스,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...
79771,서울 여자와 대구 남자가 만날 때,정민,영상노트,8100,0,0.0,,http://image.yes24.com/goods/376458/L,로맨스,


In [60]:
from math import log # IDF 계산을 위해

In [61]:
docs = df_books['genre'] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [62]:
docs.

SyntaxError: ignored

In [ ]:
N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [ ]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

In [ ]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

In [ ]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_